In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.externals import joblib
from pathlib import Path
from bio.files import download, join_all

In [3]:
# Name the place where data files will be downloaded from the CDC website
datadir = Path('../data')
# Name the file where the preprocessed data will be saved as one Pandas DataFrame
fname = datadir / 'nhanes.dat'

### Download the NHANES data files 

In [4]:
download(datadir)

Skipping ../data/LAB25.XPT (file exists)
Skipping ../data/DEMO.XPT (file exists)
Skipping ../data/LAB13.XPT (file exists)
Skipping ../data/LAB18.XPT (file exists)
Skipping ../data/LAB06.XPT (file exists)
Skipping ../data/LAB13AM.XPT (file exists)
Skipping ../data/L25_B.XPT (file exists)
Skipping ../data/L25_2_B.XPT (file exists)
Skipping ../data/DEMO_B.XPT (file exists)
Skipping ../data/L40_B.XPT (file exists)
Skipping ../data/L13AM_B.XPT (file exists)
Skipping ../data/L13_B.XPT (file exists)
Skipping ../data/L13_2_B.XPT (file exists)
Skipping ../data/L40_2_B.XPT (file exists)
Skipping ../data/L06_2_B.XPT (file exists)
Skipping ../data/L40FE_B.XPT (file exists)
Skipping ../data/L25_C.XPT (file exists)
Skipping ../data/DEMO_C.XPT (file exists)
Skipping ../data/L40_C.XPT (file exists)
Skipping ../data/L13_C.XPT (file exists)
Skipping ../data/L06COT_C.XPT (file exists)
Skipping ../data/L06BMT_C.XPT (file exists)
Skipping ../data/L06TFR_C.XPT (file exists)
Skipping ../data/L13AM_C.XPT (fil

### Concatenate and crosstabulate the files 
- It would appear NAHNES stopped collecting `Ferritin, Serum` in 2009, and `Iron Bind.Cap.(TIBC)` in 2005.
- `LDL-C` is calculated using the Friedwald equation `LDL-C=TC-HDL-(TG/5)` so there's nothing can't be learned from these other markers.
- Not sure why `MCHC` is missing in 2015.

In [5]:
df = join_all(datadir)

Checking for missing columns in 2015-2016... 
Missing columns: "Ferritin, Serum", "Iron Bind.Cap.(TIBC)", "LDL-C", "MCHC"
Checking for missing columns in 2013-2014... 
Missing columns: "Ferritin, Serum", "Iron Bind.Cap.(TIBC)"
Checking for missing columns in 2011-2012... 
Missing columns: "Ferritin, Serum", "Iron Bind.Cap.(TIBC)", "LDL-C"
Checking for missing columns in 2009-2010... 
Missing columns: "Iron Bind.Cap.(TIBC)", "LDL-C"
Checking for missing columns in 2007-2008... 
Missing columns: "Iron Bind.Cap.(TIBC)", "LDL-C"
Checking for missing columns in 2005-2006... 
Missing columns: "LDL-C"
Checking for missing columns in 2003-2004... OK
Checking for missing columns in 2001-2002... OK
Checking for missing columns in 1999-2000... OK
Done


In [6]:
# Drop duplicate rows and rows with more than 15 null columns
df = df.dropna(thresh=15).drop_duplicates()

In [7]:
# PATIENT_GNDR == 0 women, 1 == men
df.PATIENT_GNDR = df.PATIENT_GNDR.replace({2: 0})

### Summary statistics

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Lymphs (Absolute),75218.0,2.399367,1.263189,0.20,1.80,2.20,2.80,110.80
Monocytes(Absolute),75218.0,0.568671,0.208239,0.00,0.40,0.50,0.70,10.20
Neutrophils (Absolute),75218.0,4.066200,1.793694,0.10,2.80,3.80,5.00,83.10
Eos (Absolute),75218.0,0.217859,0.196255,0.00,0.10,0.20,0.30,8.40
Baso (Absolute),75218.0,0.041904,0.065696,0.00,0.00,0.00,0.10,4.70
RBC,75376.0,4.669906,0.479749,1.67,4.35,4.65,4.98,9.16
Hemoglobin,75376.0,13.815559,1.505214,5.80,12.80,13.70,14.90,19.70
Hematocrit,75376.0,40.715872,4.339273,16.30,37.60,40.40,43.80,59.90
MCV,75376.0,87.380459,6.235286,50.50,83.80,87.90,91.50,125.30
MCH,75376.0,29.654728,2.421668,13.80,28.30,29.90,31.20,74.50


In [9]:
# women
df[df.PATIENT_GNDR == 0].describe().T

,count,mean,std,min,25%,50%,75%,max
Lymphs (Absolute),38085.0,2.437091,1.112346,0.30,1.80,2.30,2.80,49.00
Monocytes(Absolute),38085.0,0.548951,0.197279,0.00,0.40,0.50,0.60,3.80
Neutrophils (Absolute),38085.0,4.195263,1.812478,0.20,2.90,3.90,5.10,22.40
Eos (Absolute),38085.0,0.198821,0.177900,0.00,0.10,0.20,0.20,5.30
Baso (Absolute),38085.0,0.043025,0.066780,0.00,0.00,0.00,0.10,4.50
RBC,38201.0,4.471133,0.397172,1.71,4.22,4.47,4.72,7.90
Hemoglobin,38201.0,13.171995,1.184549,5.80,12.50,13.20,13.90,19.70
Hematocrit,38201.0,38.874467,3.352146,16.30,36.70,38.90,41.10,59.00
MCV,38201.0,87.193780,6.264981,50.50,83.70,87.80,91.40,125.30
MCH,38201.0,29.548404,2.471034,13.80,28.30,29.80,31.20,56.20


In [10]:
# men
df[df.PATIENT_GNDR == 1].describe().T

,count,mean,std,min,25%,50%,75%,max
Lymphs (Absolute),37133.0,2.360676,1.400090,0.20,1.70,2.20,2.70,110.80
Monocytes(Absolute),37133.0,0.588897,0.217061,0.10,0.40,0.60,0.70,10.20
Neutrophils (Absolute),37133.0,3.933827,1.764466,0.10,2.80,3.70,4.80,83.10
Eos (Absolute),37133.0,0.237385,0.211679,0.00,0.10,0.20,0.30,8.40
Baso (Absolute),37133.0,0.040754,0.064546,0.00,0.00,0.00,0.10,4.70
RBC,37175.0,4.874165,0.471437,1.67,4.57,4.89,5.18,9.16
Hemoglobin,37175.0,14.476885,1.512966,6.30,13.40,14.60,15.60,19.70
Hematocrit,37175.0,42.608100,4.423423,17.90,39.40,43.10,45.90,59.90
MCV,37175.0,87.572291,6.198854,54.00,83.90,88.00,91.60,121.90
MCH,37175.0,29.763987,2.364926,15.10,28.50,29.90,31.30,74.50


### Save the data into one file

In [11]:
joblib.dump(df, open(fname, 'wb'))